# Hand Digit Recognition using Neural Network

In [3]:
import torch
import torch.nn.functional as F
import numpy as np

In [4]:
class MNISTLoader:
    def __init__(self, train_images_path, train_labels_path, test_images_path, test_labels_path):
        self.train_images_path = train_images_path
        self.train_labels_path = train_labels_path
        self.test_images_path = test_images_path
        self.test_labels_path = test_labels_path

    def load_images(self, filename):
        with open(filename, 'rb') as f:
            # Skip the magic number and dimensions (first 16 bytes)
            f.read(16)
            # Read the remaining bytes and reshape into a 28x28 array
            data = np.frombuffer(f.read(), dtype=np.uint8)
            return data.reshape(-1, 28, 28).astype(np.float32) / 255.0  # Normalize to [0, 1]

    def load_labels(self, filename):
        with open(filename, 'rb') as f:
            # Skip the magic number (first 8 bytes)
            f.read(8)
            # Read the remaining bytes as labels
            return np.frombuffer(f.read(), dtype=np.uint8)

    def load_data(self):
        train_images = self.load_images(self.train_images_path)
        train_labels = self.load_labels(self.train_labels_path)
        test_images = self.load_images(self.test_images_path)
        test_labels = self.load_labels(self.test_labels_path)

        # Flatten images to 1D
        train_images = np.reshape(train_images, (train_images.shape[0], -1))
        test_images = np.reshape(test_images, (test_images.shape[0], -1))

        return train_images, train_labels, test_images, test_labels


In [5]:
mnist_loader = MNISTLoader(
    train_images_path='C:\\Users\\Raj.Salunkhe\\Desktop\\nn from scratch\\train-images.idx3-ubyte',
    train_labels_path='C:\\Users\\Raj.Salunkhe\\Desktop\\nn from scratch\\train-labels.idx1-ubyte',
    test_images_path='C:\\Users\\Raj.Salunkhe\\Desktop\\nn from scratch\\t10k-images.idx3-ubyte',
    test_labels_path='C:\\Users\\Raj.Salunkhe\\Desktop\\nn from scratch\\t10k-labels.idx1-ubyte'
)

train_images, train_labels, test_images, test_labels = mnist_loader.load_data()

In [6]:
train_images_tensor = torch.from_numpy(train_images).clone()
train_labels_tensor = torch.from_numpy(train_labels).clone()
test_images_tensor = torch.from_numpy(test_images).clone()
test_labels_tensor = torch.from_numpy(test_labels).clone()

flattened_image_size = train_images.shape[1]
hidden_layer_size = 130

C:\Users\Raj.Salunkhe\AppData\Local\Temp\ipykernel_71032\484590563.py:2: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:212.)
  train_labels_tensor = torch.from_numpy(train_labels).clone()


# Neural Network

In [9]:
# Construction of the HIDDEN LAYER
g = torch.Generator().manual_seed(2147483647)


W1 = torch.randn((flattened_image_size, hidden_layer_size), generator=g)
b1 = torch.randn(hidden_layer_size, generator=g)

In [10]:
h = torch.tanh(train_images_tensor @ W1 + b1)

In [11]:
W2 = torch.randn((hidden_layer_size, 10), generator=g)
b2 = torch.randn(10, generator=g)

logits = h @ W2 + b2
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)

loss = F.cross_entropy(logits, train_labels_tensor)

parameters = [W1, b1, W2, b2]
print(sum(p.nelement() for p in parameters))

for p in parameters:
    p.requires_grad = True

103360


In [12]:
max_steps = 2000

losses = []
steps  = []
for i in range(max_steps):
    # Forward Pass
    h = torch.tanh(train_images_tensor @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, train_labels_tensor)

    # Backward Pass
    for p in parameters:
        p.grad = None

    loss.backward()

    lr = 1.2 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    # if i % 10000 == 0:
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')

    steps.append(i)
    losses.append(loss.item())
    

    # if loss <= 0.51:
    #     break

      0/   2000: 15.3402
      1/   2000: 10.0933
      2/   2000: 7.2498
      3/   2000: 5.9458
      4/   2000: 5.0958
      5/   2000: 4.4858
      6/   2000: 4.0879
      7/   2000: 3.7443
      8/   2000: 3.4588
      9/   2000: 3.2102
     10/   2000: 3.0052
     11/   2000: 2.8247
     12/   2000: 2.6788
     13/   2000: 2.5494
     14/   2000: 2.4417
     15/   2000: 2.3460
     16/   2000: 2.2630
     17/   2000: 2.1883
     18/   2000: 2.1214
     19/   2000: 2.0605
     20/   2000: 2.0049
     21/   2000: 1.9537
     22/   2000: 1.9063
     23/   2000: 1.8621
     24/   2000: 1.8208
     25/   2000: 1.7821
     26/   2000: 1.7456
     27/   2000: 1.7111
     28/   2000: 1.6784
     29/   2000: 1.6473
     30/   2000: 1.6178
     31/   2000: 1.5897
     32/   2000: 1.5628
     33/   2000: 1.5372
     34/   2000: 1.5126
     35/   2000: 1.4890
     36/   2000: 1.4664
     37/   2000: 1.4447
     38/   2000: 1.4238
     39/   2000: 1.4038
     40/   2000: 1.3844
     41/   200

In [13]:
import plotly.graph_objs as go

# Create a line plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=steps, y=losses, mode='lines+markers', name='Loss'))

# Add labels and title
fig.update_layout(
    title="Loss vs Steps",
    xaxis_title="Steps",
    yaxis_title="Loss"
)

fig.show()


In [10]:
# Step 1: Perform forward pass on test data
with torch.no_grad():  # We don't need gradients for evaluation
    h_test = torch.tanh(test_images_tensor @ W1 + b1)  # Hidden layer output for test data
    logits_test = h_test @ W2 + b2  # Logits for test data

# Step 2: Convert logits to probabilities (optional, if you want softmax probabilities)
counts_test = logits_test.exp()  # Exponentiate to get counts
probs_test = counts_test / counts_test.sum(1, keepdim=True)  # Normalize to get probabilities

# Step 3: Get predicted labels (the class with the maximum probability)
predicted_labels = logits_test.argmax(dim=1)

# Step 4: Calculate accuracy
correct = (predicted_labels == test_labels_tensor).sum().item()
total = test_labels_tensor.size(0)
accuracy = 100 * correct / total

print(f'Test Accuracy: {accuracy:.2f}%')


Test Accuracy: 92.01%


In [11]:
import random
from PIL import Image
import torch
from IPython.display import display

# Step 1: Randomly select 10 indices from the test set
indices = random.sample(range(len(test_images)), 50)

# Step 2: Loop through the selected indices
for index in indices:
    # Step 3: Get the image and the true label
    image = test_images[index]
    true_label = test_labels[index]

    # Step 4: Convert the image to a PIL image and display it
    pil_image = Image.fromarray((image.reshape(28, 28) * 255).astype(np.uint8))  # Convert to 0-255 range
    display(pil_image)  # Display the image using IPython

    # Step 5: Perform a forward pass on this image using the trained model
    with torch.no_grad():  # No need to compute gradients for evaluation
        image_tensor = torch.from_numpy(image).float().unsqueeze(0)  # Add batch dimension
        h_test = torch.tanh(image_tensor @ W1 + b1)  # Hidden layer output
        logits_test = h_test @ W2 + b2  # Logits output

    # Step 6: Get predicted label (the class with the highest probability)
    predicted_label = logits_test.argmax(dim=1).item()

    # Step 7: Display the predicted label
    print(f'True label: {true_label}, Predicted label: {predicted_label}\n')


True label: 6, Predicted label: 6



True label: 9, Predicted label: 9



True label: 9, Predicted label: 9



True label: 1, Predicted label: 1



True label: 1, Predicted label: 1



...

True label: 8, Predicted label: 8



True label: 6, Predicted label: 6



True label: 2, Predicted label: 2



True label: 5, Predicted label: 5



True label: 9, Predicted label: 9

